In [ ]:
# loading packages

import os
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# work dir

WORK_DIR = '../input/cassava-leaf-disease-classification'
train_labels = pd.read_csv(os.path.join(WORK_DIR, "train.csv"))

# main parameters

BATCH_SIZE = 8
STEPS_PER_EPOCH = len(train_labels) * 0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels) * 0.2 / BATCH_SIZE
EPOCHS = 20
TARGET_SIZE = 512

In [ ]:
# data

train_labels.label = train_labels.label.astype('str')

train_datagen = ImageDataGenerator(validation_split=0.2,
                                   preprocessing_function=None,
                                   rotation_range=45,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   shear_range=0.1,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_labels,
                                                    directory=os.path.join(WORK_DIR, "train_images"),
                                                    subset="training",
                                                    x_col="image_id",
                                                    y_col="label",
                                                    target_size=(TARGET_SIZE, TARGET_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="sparse")

validation_datagen = ImageDataGenerator(validation_split=0.2)
validation_generator = validation_datagen.flow_from_dataframe(train_labels,
                                                              directory=os.path.join(WORK_DIR, "train_images"),
                                                              subset="validation",
                                                              x_col="image_id",
                                                              y_col="label",
                                                              target_size=(TARGET_SIZE, TARGET_SIZE),
                                                              batch_size=BATCH_SIZE,
                                                              class_mode="sparse")

In [ ]:
# modeling

def create_model():
    model = models.Sequential()
    model.add(EfficientNetB0(include_top=False, weights='imagenet', input_shape=(TARGET_SIZE, TARGET_SIZE, 3)))
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(5, activation='softmax'))
    
    model.compile(optimizer=Adam(lr=0.001), loss="sparse_categorical_crossentropy", metrics=["acc"])
    return model


model = create_model()
model.summary()

In [ ]:
# training

model_save = ModelCheckpoint('./best_weights.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)


history = model.fit_generator(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = [model_save, early_stop, reduce_lr]
)

In [ ]:
# plot

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.set_style("white")
plt.suptitle('Train history', size = 15)

ax1.plot(epochs, acc, "bo", label = "Training acc")
ax1.plot(epochs, val_acc, "b", label = "Validation acc")
ax1.set_title("Training and validation acc")
ax1.legend()

ax2.plot(epochs, loss, "bo", label = "Training loss", color = 'red')
ax2.plot(epochs, val_loss, "b", label = "Validation loss", color = 'red')
ax2.set_title("Training and validation loss")
ax2.legend()

plt.show()

In [ ]:
model.save('./model.h5')
with open('./model.json', 'w') as outfile:
    outfile.write(model.to_json())